## **BAY AREA 2023**

# Extracción

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Definir la URL para Boston 2023 - repositorio público AWS
url = 'https://s3.amazonaws.com/baywheels-data/'

# Se solicita la información a la página y usamos BeautifulSoup para obtenre un listado estructurado en XML y extraemos los archivos .zip de 2023
response = requests.get(url)
files_2023 = [
    file.text for file in BeautifulSoup(response.text, 'xml').find_all('Key')
    if file.text.endswith('.zip') and '2023' in file.text
]

print(f"Encontrados {len(files_2023)} archivos para 2023")

Encontrados 12 archivos para 2023


# Transformaciones

### Transformación 1: Filtrado de columnas y transformación 2: Creación del campo "city"

In [ ]:
import os
import zipfile

# Definimos el nombre del archivo CSV final y las columnas requeridas
output_file = 'BayArea_2023.csv'
required_columns = ['started_at', 'ended_at', 'start_lat', 'start_lng', 'end_lat', 'end_lng', 'member_casual']

# Verificamos si el archivo de salida ya existe para no sobreescribirlo
if not os.path.exists(output_file):
    # Crear el archivo CSV inicial con las columnas seleccionadas y la columna adicional 'city' como encabezados
    pd.DataFrame(columns=required_columns + ['city']).to_csv(output_file, index=False)

# Iteramos sobre cada archivo mensual de 2023
for file_name in files_2023:
    print(f"Procesando archivo: {file_name}")

    # Descargar el archivo y descomprimir
    file_url = url + file_name
    local_zip = file_name.split('/')[-1] #nos quedamos con el nombre del mes del archivo en lugar de toda la dirección URL

    with open(local_zip, "wb") as f: #se crea un archivo vacio para meter la información de la URL de cada mes
        f.write(requests.get(file_url).content)

    # Descomprimir el archivo en una carpeta temporal
    with zipfile.ZipFile(local_zip, "r") as zip_ref: #se abre el archivo ZIP para leerlo
        zip_ref.extractall("temp_data") #se extrae la información y se añade en la carpeta temporal

    # Cargar el CSV descomprimido y filtrar por las columnas necesarias
    csv_file = [file for file in os.listdir("temp_data") if file.endswith('.csv')][0]
    df = pd.read_csv(os.path.join("temp_data", csv_file), usecols=required_columns)


    # Añadir la columna 'city' con el valor "San Francisco & San Jose" en todas las filas
    df['city'] = 'San Francisco & San Jose'

    # Guardar los datos en el archivo CSV final (añadir al archivo existente)
    df.to_csv(output_file, mode='a', index=False, header=False)

    # Limpiar archivos temporales utilizados en este apartado
    os.remove(local_zip)
    os.remove(os.path.join("temp_data", csv_file))

print(f"Archivo '{output_file}' actualizado con todos los meses de 2023 y la columna 'city' añadida.")

Procesando archivo: 202301-baywheels-tripdata.csv.zip
Procesando archivo: 202302-baywheels-tripdata.csv.zip
Procesando archivo: 202303-baywheels-tripdata.csv.zip
Procesando archivo: 202304-baywheels-tripdata.csv.zip
Procesando archivo: 202305-baywheels-tripdata.csv.zip
Procesando archivo: 202306-baywheels-tripdata.csv.zip
Procesando archivo: 202307-baywheels-tripdata.csv.zip
Procesando archivo: 202308-baywheels-tripdata.csv.zip
Procesando archivo: 202309-baywheels-tripdata.csv.zip
Procesando archivo: 202310-baywheels-tripdata.csv.zip
Procesando archivo: 202311-baywheels-tripdata.csv.zip
Procesando archivo: 202312-baywheels-tripdata.csv.zip
Archivo 'BayArea_2023.csv' actualizado con todos los meses de 2023 y la columna 'city' añadida.


In [ ]:
# Se imprime para cerciorarnos de que es el formato correcto
df = pd.read_csv('BayArea_2023.csv')
df

# Se puede observar que el formato fecha conserva la información de los milisegundos en los últimos meses del año.
#Habrá que transformarlo para que tenga el formato estándar elegido (hasta segundos)

,started_at,ended_at,start_lat,start_lng,end_lat,end_lng,member_casual,city
0,2023-01-17 16:26:07,2023-01-17 16:33:38,37.867789,-122.265896,37.871719,-122.273068,member,San Francisco & San Jose
1,2023-01-08 17:51:26,2023-01-08 18:01:11,37.755367,-122.388795,37.778588,-122.392553,casual,San Francisco & San Jose
2,2023-01-30 14:54:19,2023-01-30 15:20:18,37.765832,-122.457532,37.753259,-122.405514,member,San Francisco & San Jose
3,2023-01-19 17:10:52,2023-01-19 17:36:01,37.765804,-122.457583,37.753259,-122.405514,member,San Francisco & San Jose
4,2023-01-25 09:15:09,2023-01-25 09:20:17,37.771767,-122.386689,37.778588,-122.392553,member,San Francisco & San Jose
...,...,...,...,...,...,...,...,...
2552942,2023-12-11 17:43:58.027,2023-12-11 17:47:22.857,37.772380,-122.435964,37.764285,-122.431804,member,San Francisco & San Jose
2552943,2023-12-13 18:46:18.500,2023-12-13 18:57:05.599,37.760336,-122.418934,37.751792,-122.405216,member,San Francisco & San Jose
2552944,2023-12-05 21:51:13.971,2023-12-05 22:00:01.503,37.776661,-122.395320,37.776619,-122.417385,member,San Francisco & San Jose
2552945,2023-12-07 15:55:14.377,2023-12-07 16:03:05.120,37.776653,-122.395290,37.776619,-122.417385,member,San Francisco & San Jose


### Transformación 4: Corrección formato fecha

In [ ]:
# Reemplaza los milisegundos por vacio, solo cuando están presentes (buscando cualquier dato que tenga un punto seguido de números, esto son los milisegundos)
for col in ['started_at', 'ended_at']:
    df[col] = df[col].str.replace(r'\.\d+$', '', regex=True) #busca en las columnas los puntos (\.) seguidos de números (\d) y selecciona el final de la cadena (+$)

# Verificación rápida
df[['started_at', 'ended_at']].tail() #para comprobar que se han cambiado las últimas filas

,started_at,ended_at
2552942,2023-12-11 17:43:58,2023-12-11 17:47:22
2552943,2023-12-13 18:46:18,2023-12-13 18:57:05
2552944,2023-12-05 21:51:13,2023-12-05 22:00:01
2552945,2023-12-07 15:55:14,2023-12-07 16:03:05
2552946,2023-12-06 17:59:20,2023-12-06 18:07:34


### Transformación 5: Corrección formato tipo de cliente

Por último, también existe una inconsistencia en el formato del tipo de usuario. La columna member_casual desglosa los clientes en "member" y "casual". Sin mebargo, el resto de tablas lo hacen en "Subscriber" y "Customer". Con el objetivo de homogeneizar la información, es necesario llevar a cabo otra transformación.

In [ ]:
df['member_casual'] = df['member_casual'].replace({'member': 'Subscriber', 'casual': 'Customer'})
df['member_casual'].tail()



,member_casual
2552942,Subscriber
2552943,Subscriber
2552944,Subscriber
2552945,Subscriber
2552946,Subscriber


In [ ]:
#Guardar en csv tras las transformaciones pertinentes:
print(f"El archivo Bay Area 2023 tiene {df.shape[0]} filas y {df.shape[1]} columnas")
df.to_csv('BayArea_2023.csv', index=False)

El archivo Bay Area 2023 tiene 2552947 filas y 8 columnas
